# Design and Analyze a beam with holes

The following example demonstrates how you can add penetrations to a beam using various primitives and finally
export it to an IFC file

In [ ]:
import numpy as np
from ada import Assembly, Beam, Part, PrimBox, PrimCyl, PrimExtrude, Material
from ada.materials.metals import CarbonSteel, DnvGl16Mat
from ada.fem import Bc, FemSet, Load, StepImplicit
from ada.fem.meshing import GmshOptions
from ada.fem.utils import get_beam_end_nodes

**The basic beam setup**

The following code creates the following Assembly (and IFC) hierarchy

    MyAssembly (IfSite)
        MyPart (IfcBuildingStorey)
            MyBeam (IfcBeam)

In [ ]:
bm = Beam("MyBeam", (0, 0, 0), (1.5, 0, 0), "IPE400", Material("S420", CarbonSteel("S420", plasticity_model=DnvGl16Mat(0.1, "S420"))))
a = Assembly("MyAssembly") / [Part("MyPart") / bm]
a

**Polygon Extrusions**

In [ ]:
h = 0.2
r = 0.02

normal = [0, 1, 0]
xdir = [-1, 0, 0]


origin = np.array([0.2, -0.1, -0.1])
points = [(0, 0), (0.1, 0), (0.05, 0.1)]

poly1 = bm.add_penetration(PrimExtrude("Poly1", points, h, normal, origin, xdir))
bm

In [ ]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.05, 0.1, r)]

poly2 = bm.add_penetration(PrimExtrude("Poly2", points, h, normal, origin, xdir))
bm

In [ ]:
origin += np.array([0.2, 0, 0])
points = [(0, 0, r), (0.1, 0, r), (0.1, 0.2, r), (0.0, 0.2, r)]

poly3 = bm.add_penetration(PrimExtrude("Poly3", points, h, normal, origin, xdir))
bm

In [ ]:
# Cylinder Extrude
x = origin[0] + 0.2

cyl = bm.add_penetration(PrimCyl("cylinder", (x, -0.1, 0), (x, 0.1, 0), 0.1))
bm

In [ ]:
# Box Extrude
x += 0.2

box = bm.add_penetration(PrimBox("box", (x, -0.1, -0.1), (x + 0.2, 0.1, 0.1)))
bm

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles.ifc")
a

In [ ]:
# Create a FEM analysis of the beam as a cantilever subjected to gravity loads
p = a.get_part("MyPart")
p.fem = bm.to_fem_obj(0.1, "shell", options=GmshOptions(Mesh_MeshSizeFromCurvature=True))


a

In [ ]:
# Add a set containing ALL elements (necessary for Calculix loads).
fs = p.fem.add_set(FemSet("Eall", [el for el in p.fem.elements], "elset"))

step = a.fem.add_step(StepImplicit("gravity", nl_geom=True, init_incr=100.0, total_time=100.0))
step.add_load(Load("grav", "gravity", -9.81 * 800, fem_set=fs))

fix_set = p.fem.add_set(FemSet("bc_nodes", get_beam_end_nodes(bm), "nset"))
a.fem.add_bc(Bc("Fixed", fix_set, [1, 2, 3]))

In [ ]:
res = a.to_fem("MyCantilever_code_aster", "code_aster", execute=True, overwrite=True)

In [ ]:
res

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles.ifc")

**Add the primitives used for cutting as solid shapes to your model**

In [ ]:
a / (Part("Shapes") / [x.primitive for x in [poly1, poly2, poly3, cyl, box]])
a

In [ ]:
# Export IFC to the Home folder
a.to_ifc("../output/MyBeamWithHoles_and_Primitives.ifc")